In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import joblib 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score
from time import time
from sklearn.model_selection import GridSearchCV
import warnings
from IPython.core.interactiveshell import InteractiveShell

import statistics
reviews = []
p=[]
text_polarity=[]
accessories=[]
responserate=[]
uniqueidentifier=[]
data={}


from textblob import TextBlob



#Simple Sentiment polarity analysis


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
df=pd.read_csv('Downloads/kindle_reviews.csv')

In [ ]:
df

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,B000F83SZQ,"[0, 0]",5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J,Avidreader,Nice vintage story,1399248000
1,1,B000F83SZQ,"[2, 2]",4,This book is a reissue of an old one; the auth...,"01 6, 2014",AN0N05A9LIJEQ,critters,Different...,1388966400
2,2,B000F83SZQ,"[2, 2]",4,This was a fairly interesting read. It had ol...,"04 4, 2014",A795DMNCJILA6,dot,Oldie,1396569600
3,3,B000F83SZQ,"[1, 1]",5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",A1FV0SX13TWVXQ,"Elaine H. Turley ""Montana Songbird""",I really liked it.,1392768000
4,4,B000F83SZQ,"[0, 1]",4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",A3SPTOKDG7WBLN,Father Dowling Fan,Period Mystery,1395187200
...,...,...,...,...,...,...,...,...,...,...
44150,44150,B004KZOR8C,"[1, 2]",3,Little too much of a hero but a good readWell ...,"02 13, 2014",A1ZJY5EWNSZ33N,Leeb,Good read,1392249600
44151,44151,B004KZOR8C,"[2, 2]",5,My kind of man. Funny and exciting story line ...,"07 22, 2013",A3MGS9HQEPA07G,Mac taylor,Outstastanding,1374451200
44152,44152,B004KZOR8C,"[0, 1]",3,I was very disappointed. The book needs better...,"11 1, 2013",A2PC55KUDI44PZ,MariaVal,Not what expected,1383264000
44153,44153,B004KZOR8C,"[0, 0]",5,This book in my opinion was phenomenal!! The s...,"03 13, 2013",A7LQTXGRMVD6R,"MaryAnn ""MaryAnn""",Exciting,1363132800


In [ ]:
# np.savetxt(r'Downloads/np.txt', df.reviewText.values)

In [ ]:
with open('Downloads/reviewsKindle.txt','w') as fw:
    for review in df.reviewText:
        fw.write(str(review)+'\n')

In [ ]:
with open('Downloads/reviewsKindle.txt','r') as fw:
    revs = fw.readlines()
    from textblob import TextBlob
    count=0
    polar=0
    avg=0
    print('{:40} : {:10} : {:10}'.format("Review", "Polarity", "Subjectivity") )

    for review in revs:
        #Find sentiment of a review
        sentiment = TextBlob(review)
        #Print individual sentiments
        print('{:40} :   {: 01.2f}    :   {:01.2f}'.format(review[:40]\
                    , sentiment.polarity, sentiment.subjectivity) )
        polar+=sentiment.polarity
        count+=1
        
avg=polar/count
print("Average polarity is :")
print(avg)
text_polarity.append(avg)


        
    



reviews

data = {
    'reviews':reviews,
    'uniqueidentifier':uniqueidentifier
}

df = pd.DataFrame(data)

# check the shape of our dataframe
df.shape
Y=df['uniqueidentifier']

#Token processing just for checking
#prepare
import nltk
import os
#Download punkt package, used part of the other commands
nltk.download('punkt')

#Read the base file into a token list
file = open("Downloads/reviewsKindle.txt", 'rt')
raw_text = file.read()
#base_file.close()

#pre-processing including removing punctuations, lower case, stopwords, lemmatizer
token_list = nltk.word_tokenize(raw_text)
token_list2 = list(filter(lambda token: nltk.tokenize.punkt.PunktToken(token).is_non_punct, token_list))
token_list3=[word.lower() for word in token_list2 ]
nltk.download('stopwords')
from nltk.corpus import stopwords
token_list4 = list(filter(lambda token: token not in stopwords.words('english'), token_list3))
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
token_list5 = [lemmatizer.lemmatize(word) for word in token_list4 ]
print("\n Total Tokens : ",len(token_list5))

#build N-grams
from nltk.util import ngrams
from collections import Counter

#Find bigrams and print the most common 5
bigrams = ngrams(token_list5,2)
print("Most common bigrams : ")
print(Counter(bigrams).most_common(5))

#Find trigrams and print the most common 5
trigrams = ngrams(token_list5,3)
print(" \n Most common trigrams : " )
print(Counter(trigrams).most_common(5))

# building tf-idf matrix
#Use scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(token_list5)
X = tfidf_vectorizer.transform(token_list5)
X.toarray()

Review                                   : Polarity   : Subjectivity
I enjoy vintage books and movies so I en :    0.45    :   0.70
This book is a reissue of an old one; th :    0.28    :   0.38
This was a fairly interesting read.  It  :    0.14    :   0.59
I'd never read any of the Amy Brewster m :    0.20    :   0.20
If you like period pieces - clothing, li :    0.05    :   0.45
A beautiful in-depth character descripti :    0.24    :   0.70
I enjoyed this one tho I'm not sure why  :    0.22    :   0.64
Never heard of Amy Brewster. But I don't :    0.05    :   0.24
Darth Maul working under cloak of darkne :    0.53    :   0.45
This is a short story focused on Darth M :    0.19    :   0.41
I think I have this one in both book and :    0.70    :   0.60
Title has nothing to do with the story.  :    0.22    :   0.48
Well written. Interesting to see Sideous :    0.14    :   0.40
Troy Denning's novella Recovery was orig :    0.18    :   0.59
I am not for sure on how much of a diffe :    0.3

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/GuoJietongPRC/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/GuoJietongPRC/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/GuoJietongPRC/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



 Total Tokens :  220003
Most common bigrams : 
[(('ca', "n't"), 361), (('short', 'story'), 260), (('read', 'book'), 231), (('well', 'written'), 191), (('could', "n't"), 177)]
 
 Most common trigrams : 
[(('ca', "n't", 'wait'), 109), (("n't", 'wait', 'read'), 47), (('would', 'recommend', 'book'), 34), (('look', 'forward', 'reading'), 30), (('looking', 'forward', 'reading'), 30)]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])